In [25]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import uncertainties.unumpy as unp
import uncertainties as unc
from pint import UnitRegistry
from IPython.display import display

import analysis.util as au

In [2]:
units = UnitRegistry()

meta = dict(
    experiment='pk_exp008',
    title='ADP1 Arginine Titration Response of Arg-Auxotrophs',
    strains={
        'KSF107 ∆argB': {'∆argB', '∆astA', '∆cphAI'},
        'KSF111 ∆argB': {'∆argB', '∆argR', '∆astA', '∆cphAI'},
    },
)

# noinspection PyTypeChecker
df_od = (
    pd.read_excel(
        '~/tyo_lab_pk/04 Raw Data/EXP_0008_PK_20241009_ArgConcentrationResponse.xlsx',
        sheet_name='Day 3 - 5 OD tracking',
        usecols='A:J',
        skiprows=3,
    )
    .assign(arg_conc=lambda df: df.condition.str.split('•').str.get(0).apply(units.parse_expression))
    .assign(arg_conc_uM=lambda df: df.arg_conc.apply(lambda qty: int(qty.to('micromolar').magnitude)))
)
# df_od

In [34]:
df_od_mean_sem = (
    df_od
    .groupby(['elapsed_time_hr', 'strain_id', 'arg_conc_uM'], sort=False)
    .agg(
        od_sample_mean=pd.NamedAgg('od_sample', 'mean'),
        od_sample_sem=pd.NamedAgg('od_sample', 'sem'),
    )
    .assign(od_mean=lambda df: unp.uarray(
        nominal_values=df.od_sample_mean,
        std_devs=df.od_sample_sem,
    ))
    .drop(columns=['od_sample_mean', 'od_sample_sem'])
    .reset_index()
)
# df_od_mean_sem

In [35]:
fig_growth_raw = px.bar(
    df_od_mean_sem,
    x=df_od_mean_sem.elapsed_time_hr,
    y=unp.nominal_values(df_od_mean_sem.od_mean),
    error_y=unp.std_devs(df_od_mean_sem.od_mean),
    color='strain_id',
    facet_col='arg_conc_uM',
    barmode='group',
    labels={
        'arg_conc_uM': '[Arg] µM',
        'y': 'OD600 (mean)',
        'strain_id': 'ADP1 strain',
        'elapsed_time_hr': '',
    },
).add_traces(
    px.line(
        df_od_mean_sem,
        x=df_od_mean_sem.elapsed_time_hr,
        y=unp.nominal_values(df_od_mean_sem.od_mean),
        color='strain_id',
        facet_col='arg_conc_uM',
        markers=False,
    ).update_traces(line=dict(width=1))['data']
).update_layout(
    title=f'OD ADP1 | {au.fmt_strains(meta["strains"])} | growth in dosed Arg after preculture and wash',
    title_x=0.5,
    showlegend=True,
    boxmode='group',
    boxgap=.8,  # Gap between boxes in the same group (0.5 makes them narrower)
    boxgroupgap=0,  # Gap between different groups of boxes,
    height=500,
    xaxis_title='Elapsed Time [hr]',
).for_each_trace(au.dedup_trace_legends())

fig_growth_raw.write_image(
    f'../../figures/{meta["experiment"]}_arg_response_raw.pdf',
    width=1300,
    height=500,
)

display(fig_growth_raw)

In [36]:
# let's get the starting OD600 values (we are after the OD600 deltas)
df_od_init = df_od_mean_sem.loc[lambda df: df.elapsed_time_hr == 0].groupby(
    ['strain_id', 'arg_conc_uM']
).agg(od_init=pd.NamedAgg('od_mean', 'first'))

df_od_final = df_od_mean_sem.groupby(
    ['strain_id', 'arg_conc_uM']
).agg(od_final=pd.NamedAgg('od_mean', 'max'))

# concatenate the two series
df_od_delta = pd.concat([df_od_init, df_od_final], axis='columns').assign(
    od_delta=lambda df: df.od_final - df.od_init,
    od_fold_change=lambda df: df.od_final / df.od_init,
)

# let's subtract the background ODs
df_od_bg = df_od_delta.loc[pd.IndexSlice[:, 0], ['od_delta']].droplevel('arg_conc_uM')
df_od_delta = df_od_delta.join(df_od_bg, on='strain_id', rsuffix='_bg').assign(
    od_delta_no_bg=lambda df: df.od_delta - df.od_delta_bg,
    od_fold_change_no_bg=lambda df: (df.od_init + df.od_delta_no_bg) / df.od_init,
).reset_index()
# df_od_delta

In [37]:
df_od_delta_theoretical = pd.DataFrame(data=dict(arg_conc_uM=[0, 50])).assign(
    od=lambda df: df.arg_conc_uM * au.YIELD_OD600_FROM_X_ARG_M * 1e-6,
)
# df_od_delta_theoretical

In [38]:
px.bar(
    df_od_delta,
    x='arg_conc_uM',
    y=unp.nominal_values(df_od_delta.od_delta_no_bg),
    error_y=unp.std_devs(df_od_delta.od_delta_no_bg),
    color='strain_id',
    barmode='group',
    labels={
        'arg_conc_uM': '[Arg] µM',
        'y': '∆OD600 (sans starting and background OD)',
        'strain_id': 'ADP1 strain',
        'elapsed_time_hr': 'Elapsed Time (hr)',
    },
).update_layout(
    title=f'∆OD600 ADP1 | {au.fmt_strains(meta["strains"])}',
    title_x=0.5,
    showlegend=True,
    boxgap=1,  # Gap between boxes in the same group (0.5 makes them narrower)
    boxgroupgap=0,  # Gap between different groups of boxes,
)

In [39]:
fig_yield = px.scatter(
    df_od_delta,
    x='arg_conc_uM',
    y=unp.nominal_values(df_od_delta.od_delta_no_bg),
    error_y=unp.std_devs(df_od_delta.od_delta_no_bg),
    color='strain_id',
    labels={
        'arg_conc_uM': 'x_Arg (µM)',
        'y': '∆OD600 (au)',
        'strain_id': 'ADP1 Strain',
    },
    trendline='rolling',
    trendline_options=dict(function='median', window=1),
).update_layout(
    title=f'ADP1  Biomass/Arginine Yield Curve | {au.fmt_strains(meta["strains"])}',
    title_x=0.5,
    template='simple_white'
).add_traces([
    go.Scatter(
        x=df_od_delta_theoretical.arg_conc_uM,
        y=df_od_delta_theoretical.od,
        name='theoretical arg-auxotroph',
        mode='lines', line=dict(dash='dash'),
    ),
])

display(fig_yield)

fig_yield.write_image(
    f'../../figures/{meta["experiment"]}_yield_curve.pdf',
    width=1200,
    height=500,
)

In [40]:
yield_summary = df_od_delta.loc[lambda df: df.arg_conc_uM > 0].assign(
    yield_OD_xArg=lambda df: df.od_delta_no_bg.div(df.arg_conc_uM * 1.0e-6),
    yield_gCDW_gArg=lambda df: df.yield_OD_xArg * .55 / 174.2,
)

yield_summary_50 = yield_summary.set_index(['strain_id', 'arg_conc_uM']).loc[
    pd.IndexSlice[:, 50],
    ['od_delta_no_bg', 'od_fold_change_no_bg', 'yield_OD_xArg', 'yield_gCDW_gArg']
]
yield_summary_50
# print(yield_summary_50.to_latex(formatters={'od_delta_no_bg', '{:L}'.format}))

,,od_delta_no_bg,od_fold_change_no_bg,yield_OD_xArg,yield_gCDW_gArg
strain_id,arg_conc_uM,,,,
KSF107 ∆argB,50,0.17+/-0.08,1.31+/-0.19,(3.5+/-1.6)e+03,11+/-5
KSF111 ∆argB,50,0.31+/-0.12,1.46+/-0.22,(6.1+/-2.3)e+03,19+/-7


## Sandbox

In [12]:
def theor_arg_from_OD600_with_CGP(od600: float, frac_cgp_of_cdw: float = 0.25, frac_arg_of_cgp: float = .6,
                                  mw_arg: float = 174.2) -> float:
    return od600 * .33 * frac_cgp_of_cdw * frac_arg_of_cgp / mw_arg * 1E6


def theor_OD600_with_CGP_for_x_arg(x_arg: float, frac_cgp_of_cdw: float = 0.25, frac_arg_of_cgp: float = .6,
                                   mw_arg: float = 174.2) -> float:
    return x_arg / (.33 * frac_cgp_of_cdw * frac_arg_of_cgp / mw_arg * 1E6)


def yield_in_g_g(x):
    return x * .33 / (1.742 * 10 ** -4)


yield_in_g_g(0.0004)

theor_arg_from_OD600_with_CGP(1)
theor_OD600_with_CGP_for_x_arg(100)
theor_OD600_with_CGP_for_x_arg(250)
# t = yield_fits['KSF230'].summary().tables[1]
# t[('x1')]

0.8797979797979798

In [13]:
.506 / theor_OD600_with_CGP_for_x_arg(100, frac_cgp_of_cdw=.5)

2.8756601607347876

In [14]:
import pandera as pa
import pandera.typing as pat
from analysis.schemas import SakaguchiODSchema


@pa.check_types()
def sakaguchi_analysis(df: pat.DataFrame[SakaguchiODSchema]):
    return df

In [15]:
# noinspection PyTypeChecker
df_sakaguchi = pd.read_excel(
    '~/tyo_lab_pk/04 Raw Data/EXP_0007_PK_20240909_ArgConcentrationResponse.xlsx',
    sheet_name='Sakaguchi for residual arg',
    usecols='B:E',
    skiprows=45,
)

sakaguchi_analysis(df_sakaguchi)

SchemaError: error in check_types decorator of function 'sakaguchi_analysis': column 'conc_factor' not in dataframe. Columns in dataframe: ['sample', 'vol_sample_ul', 'vol_reagentA_ul', 'vol_reagentB_ul']

In [ ]:
plotly_colors = {
    "SP_Brights_Green": "rgb(81, 255, 0)",
    "SP_Brights_Light_Blue": "rgb(132, 255, 199)",
    "SP_Brights_Blue": "rgb(81, 168, 255)",
    "SP_Brights_Yellow": "rgb(229, 255, 28)",
    "SP_Brights_Orange": "rgb(255, 196, 13)",
    "SP_Brights_Orange_Red": "rgb(252, 45, 51)",
    "SP_Dark_Green": "rgb(27, 168, 36)",
    "SP_Dark_Light_Blue": "rgb(35, 150, 197)",
    "SP_Dark_Blue": "rgb(58, 38, 147)",
    "SP_Dark_Yellow": "rgb(209, 219, 0)",
    "SP_Dark_Orange": "rgb(201, 126, 19)",
    "SP_Dark_Orange_Red": "rgb(229, 57, 0)"
}

In [ ]:
import plotly.graph_objects as go
import numpy as np

# Create figure
fig = go.Figure()

black = 'rgba(255,255,255,0)'

# create linear space
x_vals = np.linspace(0, 10, 100)

# doubling references
y_vals_1_1 = x_vals
y_vals_1_2 = 2 * x_vals
y_vals_1_4 = 4 * x_vals

fig.add_trace(
    go.Scatter(x=x_vals, y=y_vals_1_1, mode='lines', line=dict(dash='dash', color='black', width=1), name='1:1'))
fig.add_trace(
    go.Scatter(x=x_vals, y=y_vals_1_2, mode='lines', line=dict(dash='dash', color='black', width=1), name='1:2'))
fig.add_trace(
    go.Scatter(x=x_vals, y=y_vals_1_4, mode='lines', line=dict(dash='dash', color='black', width=1), name='1:4'))

# x -> 1:1   real bad like
fig.add_trace(go.Scatter(x=np.concatenate([x_vals, x_vals[::-1]]),
                         y=np.concatenate([y_vals_1_1, np.zeros_like(x_vals)]),
                         fill='toself',
                         fillcolor=plotly_colors['SP_Brights_Orange_Red'],
                         opacity=0.2,
                         line=dict(color=black),
                         showlegend=True))
# 1:1 -> 1:2  meh
fig.add_trace(go.Scatter(x=np.concatenate([x_vals, x_vals[::-1]]),
                         y=np.concatenate([y_vals_1_1, y_vals_1_2[::-1]]),
                         fill='toself',
                         fillcolor=plotly_colors['SP_Dark_Orange_Red'],
                         opacity=0.2,
                         line=dict(color=black),
                         showlegend=False))
# 1:2 -> 1:4  ok
fig.add_trace(go.Scatter(x=np.concatenate([x_vals, x_vals[::-1]]),
                         y=np.concatenate([y_vals_1_2, y_vals_1_4[::-1]]),
                         fill='toself',
                         fillcolor=plotly_colors['SP_Dark_Green'],
                         opacity=0.2,
                         line=dict(color=black),
                         showlegend=False))
# y -> 1:4  cookin
fig.add_trace(go.Scatter(x=np.concatenate([x_vals, np.zeros_like(x_vals)]),
                         y=np.concatenate([y_vals_1_4, y_vals_1_4[::-1]]),
                         fill='toself',
                         fillcolor=plotly_colors['SP_Brights_Green'],
                         line=dict(color=black),
                         opacity=0.2,
                         showlegend=False))

fig.add_annotation(x=.9, y=.8, text="death", showarrow=False, font=dict(size=20))
fig.add_annotation(x=.8, y=.95, text="no doubling", showarrow=False, font=dict(size=20))
fig.add_annotation(x=.35, y=.95, text="1-2 doublings", showarrow=False, font=dict(size=20))
fig.add_annotation(x=.11, y=.95, text="≥2 doublings", showarrow=False, font=dict(size=20))

# theoretical and experimental curves

fig.add_trace(go.Scatter(x=x_vals, y=1.43 * x_vals, mode='lines', line=dict(color='black'), name='∆argBR Best'))
fig.add_trace(go.Scatter(x=x_vals, y=4.6 * x_vals, mode='lines', line=dict(color='purple'), name='theoretical'))

fig.add_annotation(x=.2, y=.51, text="Y theoretical = 4.5", showarrow=False, font=dict(size=16))
fig.add_annotation(x=.42, y=.4, text="Y ∆argBR (best of) = 1.5", showarrow=False, font=dict(size=16))

# Update axes and layout for the plot
fig.update_layout(
    xaxis_title='ADP1 with CGP from Phase 1 (OD600)',
    yaxis_title='ADP1 in Phase 2 (OD600)',
    xaxis=dict(range=[0, 1]),
    yaxis=dict(range=[0, 1]),
    showlegend=False,
    width=700,
    height=700,
    plot_bgcolor='rgba(0,0,0,0)'
)

fig

In [ ]:
fig.write_image('/Users/pasha/src/nu/posters/artifacts/yield_cgp_for_arg.pdf')